In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 19 18:09:25 2019

@author: mor
"""


import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report , confusion_matrix , precision_recall_curve
from imblearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score , roc_auc_score , average_precision_score, homogeneity_score, silhouette_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
import seaborn as sn
import pickle
from sklearn.cluster import DBSCAN
from nltk.corpus import stopwords
import string


In [2]:


pd.set_option('display.max_columns', 8)



In [3]:


df=pd.read_csv('data/chapter_4/enron_emails_clean.csv',index_col='Message-ID')



In [4]:


# Find all cleaned emails that contain 'sell enron stock'
mask = df['clean_content'].str.contains('sell enron stock', na=False)

# Select the data from df using the mask
print(df.loc[mask])



                                                                    From  \
Message-ID                                                                 
<6336501.1075841154311.JavaMail.evans@thyme>  ('sarah.palmer@enron.com')   

                                                                      To  \
Message-ID                                                                 
<6336501.1075841154311.JavaMail.evans@thyme>  ('sarah.palmer@enron.com')   

                                                             Date  \
Message-ID                                                          
<6336501.1075841154311.JavaMail.evans@thyme>  2002-02-01 14:53:35   

                                                                                        content  \
Message-ID                                                                                        
<6336501.1075841154311.JavaMail.evans@thyme>  \nJoint Venture: A 1997 Enron Meeting Belies O...   

                                    

In [5]:


# Create a list of terms to search for
searchfor = ['enron stock', 'sell stock', 'stock bonus','sell enron stock']

# Filter cleaned emails on searchfor list and select from df 
filtered_emails = df.loc[df['clean_content'].str.contains('|'.join(searchfor), na=False)]
print(filtered_emails)



                                                                               From  \
Message-ID                                                                            
<8345058.1075840404046.JavaMail.evans@thyme>        ('advdfeedback@investools.com')   
<1512159.1075863666797.JavaMail.evans@thyme>          ('richard.sanders@enron.com')   
<26118676.1075862176383.JavaMail.evans@thyme>                 ('m..love@enron.com')   
<10369289.1075860831062.JavaMail.evans@thyme>          ('leslie.milosevich@kp.org')   
<26728895.1075860815046.JavaMail.evans@thyme>          ('rtwait@graphicaljazz.com')   
<24716667.1075860824904.JavaMail.evans@thyme>       ('guelaguetza@club.lemonde.fr')   
<33107398.1075861369659.JavaMail.evans@thyme>          ('jlangham@kern-search.com')   
<26091852.1075860822907.JavaMail.evans@thyme>             ('jnissl@healthwise.org')   
<19319259.1075862176360.JavaMail.evans@thyme>                 ('m..love@enron.com')   
<6656919.1075860824395.JavaMail.evans@thyme

In [6]:


# Create flag variable where the emails match the searchfor terms
df['flag'] = np.where((df['clean_content'].str.contains('|'.join(searchfor), na=False) == True), 1, 0)

# Count the values of the flag variable
count = df['flag'].value_counts()
print(count)



0    1776
1     314
Name: flag, dtype: int64


In [7]:


df=df.dropna()
df=df.sample(5)



In [8]:

# Import nltk packages and string 
from nltk.corpus import stopwords
import string

# Define stopwords to exclude
stop = set(stopwords.words('english'))
stop.update(("to","cc","subject","http","from","sent", "ect", "u", "fwd", "www", "com"))

# Define punctuations to exclude and lemmatizer
exclude = set(string.punctuation)



In [9]:


# Import the lemmatizer from nltk
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()

# Define word cleaning function
def clean(text, stop):
    text = text.rstrip()
    # Remove stopwords
    stop_free = " ".join([i for i in text.lower().split() if((i not in stop) and (not i.isdigit()))])
    # Remove punctuations
    punc_free = ''.join(i for i in stop_free if i not in exclude)
    # Lemmatize all words
    normalized = " ".join(lemma.lemmatize(i) for i in punc_free.split())      
    return normalized

# Clean the emails in df and print results
text_clean=[]
for text in df['clean_content']:
    text_clean.append(clean(text, stop).split())    
print(text_clean)



[['evidently', 'experiencing', 'technical', 'difficulty', 'end', 'task', 'finished', 'shortly', 'kwiemer', 'milbank', 'pm', 'kay', 'mann', 'enron', 'e', 'next', 'generation', 'purchase', 'daed', 'ii', 'kay', 'mentioned', 'voice', 'mail', 'please', 'revise', 'signature', 'page', 'e', 'next', 'generation', 'follows', 'e', 'mail', 'back', 'wilmington', 'standing', 'execute', 'regard', 'karl', 'signature', 'block', 'e', 'next', 'generation', 'appear', 'follows', 'e', 'next', 'generation', 'llc', 'wilmington', 'trust', 'company', 'individual', 'capacity', 'solely', 'manager', 'name', 'title', 'karl', 'wiemer', 'milbank', 'tweed', 'hadley', 'mccloy', 'llp', 'one', 'chase', 'manhattan', 'plaza', 'new', 'york', 'ny', 'fax', 'kwiemer', 'milbank', 'e', 'mail', 'message', 'may', 'contain', 'legally', 'privileged', 'confidential', 'information', 'intended', 'recipient', 'employee', 'agent', 'responsible', 'delivery', 'message', 'intended', 'recipient', 'hereby', 'notified', 'dissemination', 'distr

In [10]:

fileName='data/chapter_4/text_clean.txt'

import ast

def eval_code(code):
    parsed = ast.parse(code, mode='eval')
    fixed = ast.fix_missing_locations(parsed)
    compiled = compile(fixed, '<string>', 'eval')
    return eval(compiled)

with open(fileName) as text_file:
    text_clean=eval_code(text_file.read())

def clean_str(li):
    new_text=[]
    for row in li:
        new_row=[]
        for j in row:
            if not isinstance(j, str):
                print(j)
                continue
            new_row.append(j)    
            
        new_text.append(new_row)
    return new_text

text_clean=clean_str(text_clean)



Ellipsis
Ellipsis
Ellipsis


In [11]:


# Import the packages
import gensim
from gensim import corpora

# Define the dictionary
dictionary = corpora.Dictionary(text_clean)

# Define the corpus 
corpus = [dictionary.doc2bow(text) for text in text_clean]

# Print corpus and dictionary
print(dictionary)
print(corpus)



Dictionary(5743 unique tokens: ['affiliated', 'anyone', 'attached', 'attend', 'b']...)
[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 5), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 4), (27, 2), (28, 1), (29, 1), (30, 1), (31, 3), (32, 4), (33, 1), (34, 1), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 3), (41, 2), (42, 1), (43, 1), (44, 1), (45, 4), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 1), (60, 2), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 3), (68, 7), (69, 1), (70, 2), (71, 1), (72, 2), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 3), (87, 1), (88, 1), (89, 1), (90, 2), (91, 2), (92, 2), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (

C:\Users\mor\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:


# Define the LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=5)

# Save the topics and top 5 words
topics = ldamodel.print_topics(num_words=5)

# Print the results
for topic in topics:
    print(topic)



(0, '0.026*"enron" + 0.005*"company" + 0.004*"gas" + 0.004*"need" + 0.004*"time"')
(1, '0.017*"com" + 0.011*"diabetes" + 0.010*"image" + 0.009*"message" + 0.008*"may"')
(2, '0.051*"enron" + 0.026*"ect" + 0.021*"error" + 0.012*"hou" + 0.011*"database"')
(3, '0.011*"original" + 0.011*"message" + 0.011*"com" + 0.010*"january" + 0.009*"mike"')
(4, '0.026*"enron" + 0.008*"ect" + 0.008*"stock" + 0.008*"com" + 0.006*"option"')


In [13]:


def get_topic_details(ldamodel, corpus):
    topic_details_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                topic_details_df = topic_details_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
    topic_details_df.columns = ['Dominant_Topic', '% Score', 'Topic_Keywords']
    return topic_details_df



In [14]:


# Run get_topic_details function and check the results
print(get_topic_details(ldamodel, corpus))



     Dominant_Topic  % Score  \
0               2.0   0.9954   
1               2.0   0.9899   
2               1.0   0.9490   
3               1.0   0.9971   
4               4.0   0.7989   
5               3.0   0.5016   
6               1.0   0.9988   
7               2.0   0.8620   
8               1.0   0.9972   
9               4.0   0.5015   
10              2.0   0.9975   
11              1.0   0.9808   
12              0.0   0.7578   
13              2.0   0.9934   
14              2.0   0.9752   
15              2.0   0.9958   
16              3.0   0.9980   
17              0.0   0.8825   
18              2.0   0.9991   
19              0.0   0.9491   
20              1.0   0.6395   
21              0.0   0.9891   
22              3.0   0.9975   
23              0.0   0.9745   
24              1.0   0.9379   
25              2.0   0.9888   
26              2.0   0.9934   
27              2.0   0.9958   
28              4.0   0.8835   
29              0.0   0.9949   
..      

In [15]:


# Add original text to topic details in a dataframe
contents = pd.DataFrame({'Original text':text_clean})
topic_details = pd.concat([get_topic_details(ldamodel, corpus), contents], axis=1)

# Create flag for text highest associated with topic 3
topic_details['flag'] = np.where((topic_details['Dominant_Topic'] == 3.0), 1, 0)
print(topic_details.head())



   Dominant_Topic  % Score                                     Topic_Keywords  \
0             2.0   0.9954  enron, ect, error, hou, database, com, employe...   
1             2.0   0.9899  enron, ect, error, hou, database, com, employe...   
2             1.0   0.9490  com, diabetes, image, message, may, please, en...   
3             1.0   0.9971  com, diabetes, image, message, may, please, en...   
4             4.0   0.7989  enron, ect, stock, com, option, pm, please, wo...   

                                       Original text  flag  
0  [interest, forwarded, steven, j, kean, na, enr...     0  
1  [folk, also, position, manager, saturday, mead...     0  
2  [please, check, deal, pinnacle, west, thought,...     0  
3  [louise, attached, discipline, sanction, docum...     0  
4  [lunch, barry, yesterday, suggested, might, be...     0  
